In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd
import os

In [3]:
deprel = "appos"

In [4]:
root = ".../margendatud/"+deprel
fil = root + "/" + "project-7-at-2023-04-27-05-56-c47d0051.json"
with open(fil, encoding="utf-8") as f:
    data = json.load(f)

In [5]:
fil2 = ".../train_dev_for_labelstudio/" + deprel + "/" + "ls_puupank_appos_export_big_v2_fileinfo.txt"
with open(fil2, "r", encoding="utf-8") as f:
    fileinfo = f.readlines()
    

In [6]:
data[1]

{'id': 63049,
 'annotations': [{'id': 16523,
   'completed_by': 1,
   'result': [{'value': {'start': 107,
      'end': 123,
      'text': 'Ly ja Raul Teder',
      'labels': ['appos_phrases']},
     'id': 'ixS_x2K8DB',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'origin': 'prediction'},
    {'value': {'choices': ['bound']},
     'id': 'OB5NzICfQH',
     'from_name': 'review',
     'to_name': 'text',
     'type': 'choices',
     'origin': 'manual'}],
   'was_cancelled': False,
   'ground_truth': False,
   'created_at': '2023-04-11T08:47:15.472542Z',
   'updated_at': '2023-04-11T08:47:15.472573Z',
   'lead_time': 28.709,
   'prediction': {'id': 63048,
    'model_version': 'undefined',
    'created_ago': '3\xa0minutes',
    'result': [{'to_name': 'text',
      'from_name': 'label',
      'type': 'labels',
      'value': {'start': 107,
       'end': 123,
       'text': ['Ly', 'ja', 'Raul', 'Teder'],
       'labels': ['appos_phrases']}}],
    'score': None

In [7]:
all_data = []

for obj in data:
    idx = obj["id"]
    if 'text' in obj["annotations"][0]["result"][0]["value"]:
        removed = obj["annotations"][0]["result"][0]["value"]["text"]
        if len(obj["annotations"][0]["result"]) == 1:
            print(idx)
        else:
            verdict = obj["annotations"][0]["result"][1]["value"]["choices"][0]
            sentence = obj["data"]["text"]

            remove_start = obj["annotations"][0]["result"][0]["value"]["start"]
            remove_end = obj["annotations"][0]["result"][0]["value"]["end"]
            short_sent = sentence[:remove_start] + sentence[remove_end:].strip()

            match = False
            for line in fileinfo[1:]:
                line = line.strip()
                if sentence.replace(" ", "") in line.replace(" ", "") and removed==line.split("\\")[2]:
                    fname = line.split("\\")[0]
                    cons_score = line.split("\\")[3]
                    ual = line.split("\\")[4]
                    la = line.split("\\")[5]
                    root = ".../UDpuupank/UD2_11_udreposse/"
                    if os.path.isfile(root+"Train/"+fname):
                        fpath = "Train/"+fname
                    elif os.path.isfile(root+"Dev/"+fname):
                        fpath = "Dev/"+fname

                    match = True

                    all_data.append([idx, fpath, sentence, remove_start, remove_end, removed, verdict, short_sent, cons_score, ual, la])    
                    break
        
        #if not match:
        #    print(sentence)

        #all_data.append([idx, fpath, sentence, remove_start, remove_end, removed, verdict, short_sent, cons_score, ual, la])

63457
64295


In [8]:
df = pd.DataFrame(all_data, columns=["id", "fpath", "sentence", "remove_start", "remove_end", "removed", "type", "short_sent", "cons_score", "ual", "la"])


In [9]:
df

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
0,63049,Train/aja_epl20061216_ud211.conllu,Halliste vallas Vana-Kariste külas kasvanud ta...,107,123,Ly ja Raul Teder,bound,Halliste vallas Vana-Kariste külas kasvanud ta...,92.3,92.3,92.3
1,63050,Train/aja_pm20001004_osa_3_ud211.conllu,Harju maavalitsuse pressiesindaja Kristina Män...,34,48,Kristina Männi,free,Harju maavalitsuse pressiesindaja sõnul on maa...,100.0,100.0,100.0
2,63051,Train/aja_pm20000218_osa_6_ud211.conllu,Pisas 18. veebruaril ( teistel andmetel 15. sk...,84,133,"Vincenzo , kes nimetas end Firenze härrasmeheks ,",free,Pisas 18. veebruaril ( teistel andmetel 15. sk...,95.0,95.0,100.0
3,63052,Train/aja_pm20000218_osa_5_ud211.conllu,Läti peaprokurör Janis Skrastinsh kinnitas üle...,17,33,Janis Skrastinsh,free,"Läti peaprokurör kinnitas üleeile , et Läti on...",95.5,95.5,100.0
4,63053,Train/aja_pm19980926e_ud211.conllu,Et kaks keelt ikka kindlalt kahekordselt tunne...,119,128,Juta Ütsi,bound,Et kaks keelt ikka kindlalt kahekordselt tunne...,94.1,94.1,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2744,65970,Train/aja_pm20001004_osa_1_ud211.conllu,""" See oli ekspordi vaheladu , milles oli tuhat...",62,96,", eelmise kevade ekspordi ülejääki",free,""" See oli ekspordi vaheladu , milles oli tuhat...",100.0,100.0,100.0
2745,65971,Train/aja_sloleht20071217_ud211.conllu,Loobuja oli hollandlane Robin van Aggele .,24,40,Robin van Aggele,bound,Loobuja oli hollandlane .,100.0,100.0,100.0
2746,65972,Train/aja_sloleht20071217_ud211.conllu,"Kuigi kümnest vormi selga ajanud mehest neli ,...",68,79,Travis Reed,bound,"Kuigi kümnest vormi selga ajanud mehest neli ,...",None,None,None
2747,65973,Train/aja_pm20001004_osa_3_ud211.conllu,Kolmteist aastat võitmatuna maadelnud Aleksand...,106,119,Rulon Gardner,unnatural,Kolmteist aastat võitmatuna maadelnud Aleksand...,84.6,92.3,92.3


In [10]:
df_free = df.loc[df['type'] == "free"]
df_free

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
1,63050,Train/aja_pm20001004_osa_3_ud211.conllu,Harju maavalitsuse pressiesindaja Kristina Män...,34,48,Kristina Männi,free,Harju maavalitsuse pressiesindaja sõnul on maa...,100.0,100.0,100.0
2,63051,Train/aja_pm20000218_osa_6_ud211.conllu,Pisas 18. veebruaril ( teistel andmetel 15. sk...,84,133,"Vincenzo , kes nimetas end Firenze härrasmeheks ,",free,Pisas 18. veebruaril ( teistel andmetel 15. sk...,95.0,95.0,100.0
3,63052,Train/aja_pm20000218_osa_5_ud211.conllu,Läti peaprokurör Janis Skrastinsh kinnitas üle...,17,33,Janis Skrastinsh,free,"Läti peaprokurör kinnitas üleeile , et Läti on...",95.5,95.5,100.0
5,63054,Train/tea_eesti_arst_2004_osa_2_ud211.conllu,3. Põieseinale toimivad ja detrusor'i kontrakt...,237,298,"( prazosin , terazosin , doxazosin , alfuzosin...",free,3. Põieseinale toimivad ja detrusor'i kontrakt...,93.1,96.6,93.1
6,63055,Train/aja_ee200110_osa_3_ud211.conllu,Kõige rohkem - sada miljonit krooni - annab id...,55,64,Hansapank,free,Kõige rohkem - sada miljonit krooni - annab id...,72.7,72.7,90.9
...,...,...,...,...,...,...,...,...,...,...,...
2732,65928,Train/aja_pm20000218_osa_5_ud211.conllu,"Vidinshi sõnul ei tee üks tema abidest , 19-aa...",39,56,", 19-aastane neiu",free,Vidinshi sõnul ei tee üks tema abidest seda pä...,81.8,90.9,81.8
2735,65961,Train/aja_pm20001004_osa_4_ud211.conllu,Pärnu linnavolikogu Reformierakonna fraktsioon...,215,226,Väino Linde,free,Pärnu linnavolikogu Reformierakonna fraktsioon...,91.3,91.3,91.3
2742,65968,Train/aja_ee199920_osa_2_ud211.conllu,Relva number toksiti sisse relvaregistri andme...,101,156,-- Kaitseväe Spetsiaalsete Operatsioonide Grup...,free,Relva number toksiti sisse relvaregistri andme...,86.7,100.0,86.7
2743,65969,Train/aja_ee199920_osa_2_ud211.conllu,Relva number toksiti sisse relvaregistri andme...,149,156,( SOG ),free,Relva number toksiti sisse relvaregistri andme...,85.0,95.0,90.0


In [11]:
df_free.to_csv(f"deprel_free_analysis/csv_files/{deprel}_free.csv", sep=";", index=False)

In [12]:
df_free_cons = df_free[df_free["cons_score"]=="100.0"]
df_free_cons

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
1,63050,Train/aja_pm20001004_osa_3_ud211.conllu,Harju maavalitsuse pressiesindaja Kristina Män...,34,48,Kristina Männi,free,Harju maavalitsuse pressiesindaja sõnul on maa...,100.0,100.0,100.0
10,63059,Train/aja_pm19980926e_ud211.conllu,Samuti toimetab Sven elektroonilist meelelahut...,59,125,"“ Maailma Hääl ” , millel oli suvise seisuga ü...",free,Samuti toimetab Sven elektroonilist meelelahut...,100.0,100.0,100.0
13,63062,Train/aja_ee200110_osa_3_ud211.conllu,"Ka kuuldub teatrimajast , et direktor Iljin on...",38,43,Iljin,free,"Ka kuuldub teatrimajast , et direktor on Monte...",100.0,100.0,100.0
14,63063,Train/aja_horisont2000_osa_2_ud211.conllu,Uuritakse ka rakkude etteprogrammeeritud surma...,47,58,", apoptoosi",free,Uuritakse ka rakkude etteprogrammeeritud surma .,100.0,100.0,100.0
15,63064,Train/aja_sloleht20071217_ud211.conllu,Üheksa-aastane Rapla koolipoiss Gert Karivere ...,32,45,Gert Karivere,free,Üheksa-aastane Rapla koolipoiss päästis oma vä...,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...
2698,65891,Train/aja_ee200110_osa_2_ud211.conllu,"Seal selgus , et tumesinine masin , mille külg...",85,92,Invicta,free,"Seal selgus , et tumesinine masin , mille külg...",100.0,100.0,100.0
2702,65895,Train/tea_AA_05_6_ud211.conllu,parendamine ( Improvement ),12,27,( Improvement ),free,parendamine,100.0,100.0,100.0
2711,65907,Train/aja_ee200110_osa_2_ud211.conllu,"Doktor Vask : "" Et veremaitset suus ei ole . """,7,11,Vask,free,"Doktor : "" Et veremaitset suus ei ole . """,100.0,100.0,100.0
2730,65926,Train/aja_pm19980926e_ud211.conllu,Kasutatud on IJA sekretäri Richard Dingamani s...,27,44,Richard Dingamani,free,Kasutatud on IJA sekretäri sellekohast raportit .,100.0,100.0,100.0


In [13]:
df_free_cons.to_csv(f"deprel_free_analysis/csv_files/{deprel}_free_conserved.csv", sep=";", index=False)

In [10]:
df_bound = df.loc[df['type'] == "bound"]
df_bound

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
0,63049,Train/aja_epl20061216_ud211.conllu,Halliste vallas Vana-Kariste külas kasvanud ta...,107,123,Ly ja Raul Teder,bound,Halliste vallas Vana-Kariste külas kasvanud ta...,92.3,92.3,92.3
4,63053,Train/aja_pm19980926e_ud211.conllu,Et kaks keelt ikka kindlalt kahekordselt tunne...,119,128,Juta Ütsi,bound,Et kaks keelt ikka kindlalt kahekordselt tunne...,94.1,94.1,100.0
8,63057,Train/aja_ee200110_osa_9_ud211.conllu,Need said Rootsis töötav Enno Hallek ( graafik...,154,168,Harry Liivrand,bound,Need said Rootsis töötav Enno Hallek ( graafik...,81.5,83.3,94.4
9,63058,Train/aja_ee200110_osa_9_ud211.conllu,Need said Rootsis töötav Enno Hallek ( graafik...,245,257,""" Armastan """,bound,Need said Rootsis töötav Enno Hallek ( graafik...,83.0,84.9,96.2
17,63066,Train/tea_dr8047.conllu,"E. Siguel ( 1996 ) leidis , et polüküllastumat...",83,113,üldkolesterool/HDL-kolesterool,bound,"E. Siguel ( 1996 ) leidis , et polüküllastumat...",None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
2740,65966,Train/tea_AA_05_6_ud211.conllu,Seejärel võtab ajahetkel 9 juhtimise üle prots...,83,85,w7,bound,Seejärel võtab ajahetkel 9 juhtimise üle prots...,89.7,93.1,93.1
2741,65967,Train/tea_AA_05_6_ud211.conllu,Seejärel võtab ajahetkel 9 juhtimise üle prots...,198,201,PÜ1,bound,Seejärel võtab ajahetkel 9 juhtimise üle prots...,89.7,93.1,93.1
2745,65971,Train/aja_sloleht20071217_ud211.conllu,Loobuja oli hollandlane Robin van Aggele .,24,40,Robin van Aggele,bound,Loobuja oli hollandlane .,100.0,100.0,100.0
2746,65972,Train/aja_sloleht20071217_ud211.conllu,"Kuigi kümnest vormi selga ajanud mehest neli ,...",68,79,Travis Reed,bound,"Kuigi kümnest vormi selga ajanud mehest neli ,...",None,None,None


In [11]:
df_bound.to_csv(f"deprel_free_analysis/csv_files/{deprel}_bound.csv", sep=";", index=False)

In [12]:
df_comma = df.loc[df['type'] == "redundant comma"]
df_comma

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
50,63099,Train/ilu_kivirahk_ud211.conllu,"Aga mina , vana tola , mõtlesin , et Leemet !",9,20,", vana tola",redundant comma,"Aga mina , mõtlesin , et Leemet !",100.0,100.0,100.0
111,63161,Train/aja_ee200110_osa_1_ud211.conllu,"Tema eelkäija , süngeilmeline habemik Ülo Nugi...",14,47,", süngeilmeline habemik Ülo Nugis",redundant comma,"Tema eelkäija , viibutas spiikrihaamrit nõnda ...",100.0,100.0,100.0
163,63213,Train/aja_epl20061216_ud211.conllu,"Volkswagenis sõitnud kiirabitöötaja , 52-aasta...",36,53,", 52-aastane Kaja",redundant comma,"Volkswagenis sõitnud kiirabitöötaja , toimetat...",100.0,100.0,100.0
334,63385,Train/arborest_ud211.conllu,Ent ka Rüütel ei tahtnud kogu vabastaja oreool...,104,128,putšistide erivolinikule,redundant comma,Ent ka Rüütel ei tahtnud kogu vabastaja oreool...,66.7,77.8,88.9
535,63587,Train/ilu_kivirahk_ud211.conllu,"Selles toas , kuhu ma teid praegu viin , on ne...",71,84,naisi ja mehi,redundant comma,"Selles toas , kuhu ma teid praegu viin , on ne...",73.7,78.9,84.2
547,63599,Train/aja_ee200110_osa_8_ud211.conllu,"Miks te mind , pimedat vana inimest , ära taha...",13,35,", pimedat vana inimest",redundant comma,"Miks te mind , ära tahate eksitada ! """,88.9,88.9,88.9
549,63601,Train/aja_horisont2000_osa_2_ud211.conllu,"Nanak , sünni poolest hindu , hakkas õpetama ,...",6,27,", sünni poolest hindu",redundant comma,"Nanak , hakkas õpetama , et “ pole olemas hind...",72.7,77.3,86.4
550,63602,Train/ilu_maailm_ud211.conllu,""" Aga Jooga , see türakummardaja , küll . """,12,32,", see türakummardaja",redundant comma,""" Aga Jooga , küll . """,100.0,100.0,100.0
579,63632,Train/aja_luup200202_osa_2_ud211.conllu,"Noorim kolmest tütrest , Alija , pandi aga 199...",23,30,", Alija",redundant comma,"Noorim kolmest tütrest , pandi aga 1998. aasta...",87.5,93.8,87.5
702,63757,Train/arborest_ud211.conllu,"Ka kommunistlikus ühiskonnas , kõikesuutvate m...",31,61,kõikesuutvate masinate ajastul,redundant comma,"Ka kommunistlikus ühiskonnas , , jääb püsima f...",80.0,80.0,100.0


In [13]:
df_comma.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_comma.csv", sep=";", index=False)

In [14]:
df_un = df.loc[df['type'] == "unnatural"]
df_un

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
1075,64134,Train/aja_ee200110_osa_4_ud211.conllu,"Ajakiri Seltskond kirjutas , et eurolaulu võid...",61,69,Nightman,unnatural,"Ajakiri Seltskond kirjutas , et eurolaulu võid...",85.7,85.7,85.7
1893,64957,Train/aja_ml200247_osa_4_ud211.conllu,Kommenteerib välisminister Kristiina Ojuland :...,27,44,Kristiina Ojuland,unnatural,Kommenteerib välisminister : Suurkohtumine ise...,20.0,26.7,26.7
2438,65543,Train/aja_ee200110_osa_8_ud211.conllu,Diskoteek Patarei tegutses üheksakümnendate al...,10,17,Patarei,unnatural,Diskoteek tegutses üheksakümnendate alguses Li...,91.7,91.7,91.7
2747,65973,Train/aja_pm20001004_osa_3_ud211.conllu,Kolmteist aastat võitmatuna maadelnud Aleksand...,106,119,Rulon Gardner,unnatural,Kolmteist aastat võitmatuna maadelnud Aleksand...,84.6,92.3,92.3


In [15]:
df_un.to_csv(f"deprel_free_analysis/csv_files/{deprel}_unnatural.csv", sep=";", index=False)

In [16]:
df_dub = df.loc[df['type'] == "dubious"]
df_dub

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la


In [17]:
df_dub.to_csv(f"deprel_free_analysis/csv_files/{deprel}_dubious.csv", sep=";", index=False)

In [18]:
df_red = df.loc[df['type'] == "other redundant punctuation"]
df_red

,id,fpath,sentence,remove_start,remove_end,removed,type,short_sent,cons_score,ual,la
84,63133,Train/aja_ml200247_osa_8_ud211.conllu,Tallinna lähedusel on omad miinused : näiteks ...,38,99,"näiteks pealinnast levivad pahed , kuritegevus...",other redundant punctuation,Tallinna lähedusel on omad miinused : .,85.7,100.0,85.7
104,63154,Dev/arborest_dev_ud211.conllu,Lennart ( 67 ) on nagu prints valgel hobusel -...,47,126,"peen mees musta kaabu ja valge siidsalliga , k...",other redundant punctuation,Lennart ( 67 ) on nagu prints valgel hobusel - .,45.5,45.5,81.8
243,63293,Train/aja_ee200110_osa_9_ud211.conllu,Ohtlik suvi - Läti film 74 351,14,30,Läti film 74 351,other redundant punctuation,Ohtlik suvi -,100.0,100.0,100.0
415,63467,Train/tea_toohoive.conllu,Et olukorda tööhõive osas vähegi kontrolli all...,217,273,toiduainete tööstuses ja meelelahutus- ja turi...,other redundant punctuation,Et olukorda tööhõive osas vähegi kontrolli all...,96.7,96.7,100.0
471,63523,Train/aja_luup200202_osa_2_ud211.conllu,Moodne aeg lubab küll Hollandi printsessil ( t...,81,92,kuningannal,other redundant punctuation,Moodne aeg lubab küll Hollandi printsessil ( t...,88.9,94.4,91.7
568,63621,Train/tea_toohoive.conllu,"Töö "" koguhulga "" vähenemisele perioodil 1989-...",76,311,näiteks keskmine nädala töötundide arv ja kõrv...,other redundant punctuation,"Töö "" koguhulga "" vähenemisele perioodil 1989-...",50.0,50.0,58.3
715,63770,Train/aja_pm19980926e_ud211.conllu,Riik on jagatud 51-ks ametliku staatusega vein...,113,115,DO,other redundant punctuation,Riik on jagatud 51-ks ametliku staatusega vein...,100.0,100.0,100.0
722,63777,Train/arborest_ud211.conllu,"See kujutas vaid üht tohutut , enam kui meetri...",60,159,umbes neljakümne viie aastase mehe nägu tiheda...,other redundant punctuation,"See kujutas vaid üht tohutut , enam kui meetri...",66.7,66.7,91.7
748,63803,Train/tea_AA_05_6_ud211.conllu,3. RDF- Resource Description Framework,8,38,Resource Description Framework,other redundant punctuation,3. RDF-,50.0,50.0,50.0
852,63908,Train/aja_luup200106_osa_6_ud211.conllu,Ka kasutaja ise võib õpetada oma e-posti progr...,105,136,eemalda ( saajate nimekirjast ),other redundant punctuation,Ka kasutaja ise võib õpetada oma e-posti progr...,95.0,95.0,95.0


In [19]:
df_red.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_punct.csv", sep=";", index=False)

In [ ]:
df_bound = df.loc[df['type'] == "bound"]
df_bound

df_bound.to_csv(f"deprel_free_analysis/csv_files/{deprel}_bound.csv", sep=";", index=False)

df_comma = df.loc[df['type'] == "redundant comma"]
df_comma

df_comma.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_comma.csv", sep=";", index=False)

df_un = df.loc[df['type'] == "unnatural"]
df_un

df_un.to_csv(f"deprel_free_analysis/csv_files/{deprel}_unnatural.csv", sep=";", index=False)

df_dub = df.loc[df['type'] == "dubious"]
df_dub

df_dub.to_csv(f"deprel_free_analysis/csv_files/{deprel}_dubious.csv", sep=";", index=False)

df_red = df.loc[df['type'] == "other redundant punctuation"]
df_red

df_red.to_csv(f"deprel_free_analysis/csv_files/{deprel}_redundant_punct.csv", sep=";", index=False)

## Vabade laienditega laused conll formaadis failidesse

In [11]:
from estnltk.converters.conll.conll_exporter import sentence_to_conll
from estnltk import Text
from estnltk.taggers import WhiteSpaceTokensTagger
from estnltk.taggers import PretokenizedTextCompoundTokensTagger
from estnltk_neural.taggers.syntax.stanza_tagger.stanza_tagger import StanzaSyntaxTagger

2023-06-14 08:19:19.300325: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 08:19:19.839078: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-14 08:19:21.757544: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-14 08:19:21.757794: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [12]:
model_path=".../lib/python3.9/site-packages/estnltk_neural/taggers/syntax/stanza_tagger/stanza_resources/"
input_type="morph_extended"
stanza_tagger = StanzaSyntaxTagger(input_type=input_type, input_morph_layer=input_type, add_parent_and_children=True, resources_path=model_path)

In [13]:
tokens_tagger = WhiteSpaceTokensTagger()
compound_tokens_tagger = PretokenizedTextCompoundTokensTagger()

In [14]:
from tqdm import tqdm

In [15]:
shortened_sents_data_dev = []
original_sents_data_dev = []
shortened_sents_data_train = []
original_sents_data_train = []

for row in tqdm(df_free.iterrows()):

    file = row[1][1]
    
    folder = file.split("/")[0]
    #print(folder)
    #break
    
    with open(f".../UDpuupank/UD2_11_udreposse/{file}") as f:
        conll = f.readlines()
        
    removed = row[1][5].split(" ")
    remove_start = row[1][3]
    remove_end = row[1][4]
    original_sent = row[1][2]
    sent = row[1][2].replace(" ", "")

    important_data = []

    # collect rows related to this sentence
    start = False
    for i, line in enumerate(conll):
        line = line.strip()
        mod_line = line.replace(" ", "")
        if start and not line.startswith("#"):
            important_data.append(line)
        elif start and (line=="" or line.startswith("#")):
            break    
        elif line.startswith("#") and sent in mod_line:
            start = True
            important_data.append(conll[i-1].strip())
            important_data.append(line)
            
            
    
                
            
    
    # find lines with removed words
    if len(important_data) != 0:
        
        #if "sent_id" in important_data[0]:
        #    sent_id = important_data[0].split("=")[1].strip()
        #    if sent_id in sent_dict.keys():
        if folder == "Train":
            original_sents_data_train.append(important_data)    
        elif folder == "Dev":
            original_sents_data_dev.append(important_data)
        else:
            print(f"folder {file} problem")
            break
        
        
        short_data_train = []
        short_data_dev = []
        
        short_data_pre = []
        cut_word_ids = []
        short_data_pre.append(important_data[0].strip())

        remove_count = 0
        for line in important_data[2:]:
            #print(line)

            checked = False  
            if line != "" and "sent_id =" not in line and "text =" not in line:
                added = len(line.split("\t")[1].strip() + " ")
                
                if line.split("\t")[1] in removed:
                    if len(cut_word_ids)==0 and remove_count == remove_start:
                        cut_word_ids.append(line.split("\t")[0])
                    elif len(cut_word_ids)!=0 and remove_count <= remove_end:
                        cut_word_ids.append(line.split("\t")[0])
                    checked = True
                remove_count += added
            #if not checked:    
            #    short_data.append(line+"\n")

        short_sentence = original_sent[:remove_start] + original_sent[remove_end+1:]

        txt = Text(short_sentence)
        
        tokens_tagger.tag(txt)
        compound_tokens_tagger.tag(txt)
        txt.tag_layer("morph_extended")
        stanza_tagger.tag(txt)
        res = sentence_to_conll(txt.sentences[0], txt["stanza_syntax"], udpipe=True)
        res2 = res.split("\n")
        
        
        short_data_pre.append("# cut_word_ids = " + ",".join(cut_word_ids))
        short_data_pre.append('# ud_version = 2.11')
        short_sent_text = "# text = " + " ".join([span.text for span in txt.words])
        short_data_pre.append(short_sent_text)
        
        
        #if "sent_id" in important_data[0]:
        #    sent_id = important_data[0].split("=")[1].strip()
        #    if sent_id in sent_dict.keys():
        if folder == "Train":
            short_data_train = short_data_pre + res2
            shortened_sents_data_train += [short_data_train]    
        elif folder == "Dev":
            short_data_dev = short_data_pre + res2
            shortened_sents_data_dev += [short_data_dev]
        
    #print(original_sent)
    #print(important_data)
    #print("\n",short_sent_text)
    #print(sentence_to_conll(row[1][7][:-2] + row[1][7][-2:].strip()))
    
    #print(short_data)
    
    #break
    
    
    #else:
    #    print("PROBLEEMIGA:", row[1][2])
    
#shortened_sents_data




1107it [04:50,  3.81it/s]


In [16]:
sents_to_file = 2000

left = len(shortened_sents_data_dev) % sents_to_file
files = int(len(shortened_sents_data_dev)/sents_to_file)
if left != 0:
    files += 1

for i in range(files):
    with open(f"deprel_free_analysis/Dev/{deprel}_cut.conllu", "w", encoding="utf-8") as f:
        for obj in shortened_sents_data_dev[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
    with open(f"deprel_free_analysis/Dev/{deprel}_original.conllu", "w", encoding="utf-8") as f:
        for obj in original_sents_data_dev[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
                
left = len(shortened_sents_data_train) % sents_to_file
files = int(len(shortened_sents_data_train)/sents_to_file)
if left != 0:
    files += 1                
for i in range(files):
    with open(f"deprel_free_analysis/Train/{deprel}_cut.conllu", "w", encoding="utf-8") as f:
        for obj in shortened_sents_data_train[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")
                
    with open(f"deprel_free_analysis/Train/{deprel}_original.conllu", "w", encoding="utf-8") as f:
        for obj in original_sents_data_train[i*sents_to_file:(i+1)*sents_to_file]:
            for line in obj:
                f.write(line)
                f.write("\n")